In [1]:
cd ..

/home/sumgh/Data Science/projects/Text-Summarizer


## Create the config.yml

data_transformation:
  root_dir: artifacts/data_transformation
  data_path: artifacts/data_ingestion/samsum_dataset
  tokenizer_name: google/pegasus-cnn_dailymail
  output_path: artifacts/data_transformation/samsum_dataset

## Ceate the Data Transformation Entity

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path
    output_path: Path

## Create the Data Transformation Config Manager

In [3]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path = config.data_path,
            tokenizer_name=config.tokenizer_name,
            output_path=config.output_path
        )

        return data_transformation_config

## Create the Data Transformation Component

In [4]:
import os
from textSummarizer.logger import logging
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(self.config.output_path)


/home/sumgh/Data Science/projects/Text-Summarizer/ts_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[ 2024-05-11 15:18:55,828 ] :: INFO :: config :: 58 :: [ PyTorch version 2.3.0 available. ]


## Create the Data Data Transformation Pipeline

In [6]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[ 2024-05-11 15:18:56,154 ] :: INFO :: common :: 29 :: [ yaml file: config/config.yaml loaded successfully ]
[ 2024-05-11 15:18:56,157 ] :: INFO :: common :: 29 :: [ yaml file: params.yaml loaded successfully ]
[ 2024-05-11 15:18:56,158 ] :: INFO :: common :: 49 :: [ created directory at: artifacts ]
[ 2024-05-11 15:18:56,160 ] :: INFO :: common :: 49 :: [ created directory at: artifacts/data_transformation ]


/home/sumgh/Data Science/projects/Text-Summarizer/ts_env/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/home/sumgh/Data Science/projects/Text-Summarizer/ts_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 68643.53 examples/s]
